## Import Packages

In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from multiprocessing.pool import ThreadPool
from tqdm import tqdm
import time 
from time import sleep
from multiprocessing import Pool
# fr⁄om progressbar import progressbar

import warnings
warnings.filterwarnings('ignore')

## Read the data 

In [2]:
filename = 'data/raw_da_qs.csv'

data_total = pd.read_csv(filename)

data_total.head()

,year,month,day,url,title,letterId,question_only
0,1985,1.0,01,proquest,WOMAN NEEDS HELP: HER BURDEN OF HOPELESSNESS I...,1,i have been in a bad marriage for 40 years. i ...
1,1985,1.0,01,proquest,WOMAN NEEDS HELP: HER BURDEN OF HOPELESSNESS I...,1,"this is for all newspaper carriers, mail carri..."
2,1985,1.0,02,proquest,LAMENT ABOUT OLD AGE AND SICKNESS IS ILL-ADVISED,1,our 16-year-old son recently was placed in a s...
3,1985,1.0,03,proquest,'NORMAL' WIDOW HAS AN EYE FOR THE GUYS,1,"i was a happy, respectable wife for 40 years, ..."
4,1985,1.0,04,proquest,IT'S WISE TO BE WARY WHEN A STRANGER COMES TO ...,1,you be the judge: last night about 7 p.m. i wa...


### Replace the month NAN with the median value

In [3]:
data_total.month.isna().sum(), np.isnan(data_total.month).median()

(3, 0.0)

In [4]:
def replace_month_nan(df = data_total):
    for i in range(len(df)):
        if np.isnan(df.month[i]):
            df.month[i] = np.isnan(df.month).median()
    return df

replace_month_nan()

data_total.month.isna().sum()

0

### Create the 'DATE' column

In [5]:
def get_date(df=data_total):
    df['date'] = ''*len(df)
    
    for row in range(len(df)):
        df['date'][row] = str(df['day'][row]) + '-' + str(df['month'][row]) + '-' + str(df['year'][row])
    
    return df[['url','question_only','date']]

data_total = get_date()

## Instantiate Model

In [6]:
data_total.head(3)

,url,question_only,date
0,proquest,i have been in a bad marriage for 40 years. i ...,01-1.0-1985
1,proquest,"this is for all newspaper carriers, mail carri...",01-1.0-1985
2,proquest,our 16-year-old son recently was placed in a s...,02-1.0-1985


In [7]:
#instantiate the model
tokenizer = AutoTokenizer.from_pretrained("google/roberta2roberta_L-24_bbc")
model = AutoModelForSeq2SeqLM.from_pretrained("google/roberta2roberta_L-24_bbc")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


### Create summarization function

In [ ]:
for i in range(2):
    text = data_total.question_only[i]
    print(text)
    
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids)[0]
    print("\t\t\t\t", tokenizer.decode(output_ids, skip_special_tokens=True, max_length=20))

In [ ]:
data_total = data_total.dropna()
data_total.info()

In [ ]:


data_total['question_summary'] = ''*len(data_total)

def summarize_questions(range_ab ,df = data_total):
    r1,r2 = range_ab[0],range_ab[1]
    # bar = progressbar.ProgressBar(mval=len(r2-r1)).start()
    
    if r2 is None :
        r2 = len(df)
        
    for row in tqdm(range(r1,r2)):
        input_ids = tokenizer.encode(df.question_only[row],
                                     return_tensors='pt', 
                                     padding=True, 
                                     truncation=True, 
                                     max_length=20, 
                                     add_special_tokens = True)
        output_ids = model.generate(input_ids)[0]
        res = tokenizer.decode(output_ids,skip_special_tokens=True)
        
        df['question_summary'][row] = res
        sleep(0.02)
        # bar.update(idx)
        
        # print(" Process %s waiting %s seconds" % (row, row+1))
        # time.sleep(int(row+1))
        # print(" Process %s Finished." % row)
    
    return df

In [ ]:
a = list(range(0, 20100, 100))
b = list(range(100, 20100, 100))

In [ ]:
ranges = list(zip(a, b))
# ranges, len(ranges)

In [ ]:
pool = ThreadPool()

results = pool.map(summarize_questions, ranges)

pool.close()
pool.join()

In [ ]:
display(data_total, data_total.question_summary.isnull().count())